In [ ]:
## Configuration
import os
import pandas 
import pandas as pd
import nibabel as nib
import numpy as np
import scipy.stats as stats
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from scipy.spatial import KDTree
from sklearn.metrics import pairwise_distances
from sklearn import model_selection, linear_model
import scipy.io as io
import fastparquet
import snappy
import subprocess
import random
import copy
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')


In [ ]:
import os
import time
import cupy as cp
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
# import torch
start = time.time()
mempool = cp.get_default_memory_pool()

# 1.0 Gene*Samples matrix of the cerebellum
3466x15633 to 337x15624

In [ ]:
# Default directory
atlas = '/n02dat01/users/ypwang/Gradient/GeneticGradient/Data/Cerebellum-MNIfnirt-maxprob-thr25.nii'
aba_dir = '/n01dat01/ypwang/Software/abagen/OriginalData_copy/'
data_dir = '/n02dat01/users/ypwang/Gradient/GeneticGradient/Data'
fig_dir = '/n02dat01/users/ypwang/Gradient/GeneticGradient/Fig/PLSR_2022.09.12/'                                      


In [ ]:
#_*_coding:utf-8_*_
import sys
sys.path.append('/n02dat01/users/ypwang/Gradient/GeneticGradient/Scripts/self_corrected_library/')



In [ ]:
%load_ext autoreload
%autoreload 1
%aimport abagen_cere.abagen.allen_cere
from abagen_nosuit.abagen import allen
# from abagen_cere.abagen import allen_cere

In [ ]:
# # allen_cere.get_samples_in_mask_annot
# # allen.get_samples_in_mask_annot

expression_all, report_all = allen.get_samples_in_mask_annot(data_dir=aba_dir,   
                                                             corrected_mni=True, 
                                                             return_report=True,
                                                             Allsample_nodrop='noB')

In [ ]:
# report = pandas.DataFrame(report_all[report_all.slab_type=='CB'], copy=True)
# report = pandas.DataFrame(report[-report.structure_name.str.contains('nucleus')], copy=True) # From 368*18 to 337*18
# expression = expression_all[expression_all.index.isin(report.index)]
# report.head()

In [ ]:
# # Read in the corrected cordinates
# coord_pth = '/n02dat01/users/ypwang/Gradient/AHBA_SampleCorrrected/Donor/'
# sasheets = sorted(glob(os.path.join(coord_pth,'*/*_mni_coordinates.csv')))
# ref = []
# for sheet in sasheets:
#     did = sheet.split('/')[-2]
#     sa = pandas.read_csv(sheet, header=None)
#     sa.columns = ['suit_x', 'suit_y', 'suit_z']
#     sa.index = report[report.donor==did].index
#     ref.append(sa)
# SA = pandas.concat(ref)
# report = pandas.merge(report, SA, left_index=True, right_index=True)

In [ ]:
# pandas.DataFrame(expression).to_csv(os.path.join(data_dir,'expression_cere_337.csv'))
# pandas.DataFrame(report).to_csv(os.path.join(data_dir,'report_cere_337.csv'))

# 2.0 FG of each cerebellar sample

In [ ]:
# def disbtw(surf, aa):
#     surf= np.array(surf)
#     aa = np.expand_dims(aa,axis=0)
#     dis = np.sqrt(((surf-aa)**2).sum(axis=1))
#     return dis

# def Grid2world(GridList, affine):
#     """
#     trans the grid coords into the corresponding MNI coords in MNI space
#     GridList shape: N*3, affine shape 4*4
#     """
#     GridList = np.array(GridList)
#     length = len(GridList)
#     affine = np.array(affine)
#     Grid_coord = np.concatenate([GridList, np.ones([length, 1])], axis=1)
#     MNI_coord = Grid_coord.dot(affine.T)      
#     MNI_coord = MNI_coord.round(2)
#     return MNI_coord[:, :3]
# expression = pandas.read_csv(os.path.join(data_dir,'expression_cere_337.csv'), index_col=0)
# report = pandas.read_csv(os.path.join(data_dir,'report_cere_337.csv'), index_col=0)
# loclist = report[['suit_x','suit_y','suit_z']].values.astype('float')

# Rad_dir = '/n02dat01/users/ypwang/Gradient/GeneticGradient/Data/BestRad/'
# xpsheets_name = '/n02dat01/users/ypwang/Gradient/GeneticGradient/Data/FG_noBrainSmash/result_cere2extra_gradient*__z_17853_nifti.nii'
# xpsheets = sorted(glob(xpsheets_name))
# y_gradient = {}
# for sheet in xpsheets: 
#     fid = sheet.split('/')[-1].split('_')[-5] # get functional gradient id
#     print(fid)
#     gradient_file = nib.load(sheet)
#     gradient = gradient_file.get_data()
#     affine = gradient_file.affine
#     grid_list = np.nonzero(gradient)   
#     grad_value = gradient[grid_list] 
#     rrlist = []
#     MNI_loc = Grid2world(np.array(grid_list).T, affine)
#     for loc in loclist:
#         dis = disbtw(MNI_loc, loc)
#         tmp = grad_value[dis<4].mean()
#         rrlist.append(tmp)
#     # print(np.array(rrlist))
#     filename = 'Rad4mm_' + str(fid) + '_sample337_cere2extra.csv'
#     pd.DataFrame(rrlist).to_csv(os.path.join(Rad_dir,filename))

In [ ]:
# y_gradient = {}
# xpsheets = sorted(glob('/n02dat01/users/ypwang/Gradient/GeneticGradient/Data/BestRad/Rad4mm_*_sample337_cere2extra.csv'))
# for sheet in xpsheets:
#     fid = sheet.split('/')[-1].split('_')[1] 
#     y_gradient[fid] = pandas.DataFrame(pandas.read_csv(sheet, index_col=0))
#     y_gradient[fid].index = report.index
#     report[fid] = np.NaN
#     for x in report.index:
#         report.loc[x,fid] = y_gradient[fid].loc[x].values
#     y_gradient[fid] = y_gradient[fid].values.tolist()
# pandas.DataFrame(expression).to_csv(os.path.join(data_dir,'expression_suit4mm_337_cere2extra.csv'))
# pandas.DataFrame(report).to_csv(os.path.join(data_dir,'report_suit4mm_337_cere2extra.csv'))

In [ ]:
# expression = pandas.read_csv(os.path.join(data_dir,'expression_suit4mm_337_cere2extra.csv'), index_col=0)
# report = pandas.read_csv(os.path.join(data_dir,'report_suit4mm_337_cere2extra.csv'), index_col=0)
# report.index = expression.index
# report_cur = report[~np.isnan(report['gradient1'])]
# expression_cur = expression[~np.isnan(report['gradient1'])]
# report = report_cur.iloc[np.where(report_cur['gradient1']!=0)]
# expression = expression_cur.iloc[np.where(report_cur['gradient1']!=0)]
# pandas.DataFrame(expression).to_csv(os.path.join(data_dir,'expression_4mm_317_cere2extra.csv'))
# pandas.DataFrame(report).to_csv(os.path.join(data_dir,'report_4mm_317_cere2extra.csv'))

# 3.0 PLSR: Gene to predict FG

In [ ]:
git_dir = '/n02dat01/users/ypwang/Gradient/GeneticGradient/Scripts/'
import sys
sys.path.insert(0,git_dir)

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport FunctionLibrary

In [ ]:
expression = pandas.read_csv(os.path.join(data_dir,'expression_4mm_317_cere2extra.csv'), index_col=0)
report = pandas.read_csv(os.path.join(data_dir,'report_4mm_317_cere2extra.csv'), index_col=0)

## 3.1 Use 10 cv to confirm the optimal component number

color

In [ ]:
# Color bar
from PIL import Image, ImageDraw, ImageFont


def get_dominant_colors(infile):
    image = Image.open(infile)
    
    # 缩小图片，否则计算机压力太大
    small_image = image.resize((80, 80))
    result = small_image.convert(
        "P", palette=Image.ADAPTIVE, colors=10
    )  
	
	# 10个主要颜色的图像

    # 找到主要的颜色
    palette = result.getpalette()
    color_counts = sorted(result.getcolors(), reverse=True)
    colors = list()

    for i in range(10):
        palette_index = color_counts[i][1]
        dominant_color = palette[palette_index * 3 : palette_index * 3 + 3]
        colors.append(tuple(dominant_color))

    # print(colors)
    return colors

def RGB_to_Hex(rgb):
    RGB = rgb.split(',')
    color = '#'
    for i in RGB:
        num = int(i)
        color += str(hex(num))[-2:].replace('x', '0').upper()
    return color


image_path = "/n02dat01/users/ypwang/Gradient/GeneticGradient/Data/FG_noBrainSmash/result_cere2extra_gradient1__z_17853.png"
color_1 = get_dominant_colors(image_path)
print(color_1)
color_rgb = [str(x).replace('(','').replace(')','')  for x in color_1]
Hex_1 = [RGB_to_Hex(x) for x in color_rgb]

image_path = "/n02dat01/users/ypwang/Gradient/GeneticGradient/Fig/Color/红包配色.jpg"
color_2 = get_dominant_colors(image_path)
print(color_2)
color_rgb = [str(x).replace('(','').replace(')','')  for x in color_2]
Hex_2 = [RGB_to_Hex(x) for x in color_rgb]
sns.palplot(Hex_1)
sns.palplot(Hex_2)

from matplotlib import cm
jet_10 = cm.get_cmap('jet', 10)(np.arange(10))
cm.get_cmap('jet', 10)

jet_9 = cm.get_cmap('jet', 9)(np.arange(9))
cm.get_cmap('jet', 9)

RdBu_9 = cm.get_cmap('RdBu', 9)(np.arange(9))
cm.get_cmap('RdBu', 9)

In [ ]:
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, r2_score

In [ ]:
import multiprocessing
import scipy.stats as stats
import time
from tqdm import tqdm
start = time.time()
def add(x, y):
	return x+y
def merge_add(args):
	return add(*args)


In [ ]:
def Component_eval_10cv(times):
    cv=10
    cv_inner=10
    y = report['gradient1']
    best_comps = []
    r = []
    preds = pandas.DataFrame(np.zeros((len(y))))
    preds.index = expression.index
    ncomps=np.arange(1,11)
    sel = model_selection.KFold(n_splits=cv, shuffle=True, random_state=(123 * (times+1)))    
    sel_inner = model_selection.KFold(n_splits=cv_inner, shuffle=True, random_state=(123 * (times+1)))
    for tr_ind, te_ind in sel.split(expression):
        mse=[]
        expression_cur = expression.iloc[tr_ind,:]
        for tr_i, te_i in sel_inner.split(expression_cur):
            mse.append([])
            tr_cols = expression.index[tr_ind][tr_i]
            tr_set = pandas.DataFrame(expression.loc[tr_cols,:], copy=True)
            tr_y = pandas.DataFrame(report['gradient1']).loc[tr_cols,:]
            te_cols = expression.index[tr_ind][te_i]
            te_set = pandas.DataFrame(expression.loc[te_cols,:], copy=True)
            te_y = pandas.DataFrame(report['gradient1']).loc[te_cols,:]
            for nc in ncomps:
                clf = PLSRegression(n_components=nc)
                pred_y = clf.fit(tr_set,tr_y).predict(te_set).flatten()
                mse[-1].append(mean_squared_error(te_y,pred_y))
        mse = np.sum(mse,axis=0)
        nc = ncomps[np.argmin(mse)]
        best_comps.append(nc)
        #refit full model
        clf = PLSRegression(n_components=nc)
        tr_cols = expression.index[tr_ind]
        tr_set = pandas.DataFrame(expression.loc[tr_cols,:], copy=True)
        tr_y = pandas.DataFrame(report['gradient1']).iloc[tr_ind,:]
        te_cols = expression.index[te_ind]
        te_set = pandas.DataFrame(expression.loc[te_cols,:], copy=True)
        te_y = pandas.DataFrame(report['gradient1']).iloc[te_ind,:]
        mod = clf.fit(tr_set,tr_y)
        preds.loc[te_cols,:] = mod.predict(te_set)
        r.append(stats.pearsonr(mod.predict(te_set).flatten(), te_y)[0])
    return best_comps,r,preds

In [ ]:
# cv=10
# cv_inner=10
# y = report['gradient1']
# best_comps = []
# r = []
# preds = pandas.DataFrame(np.zeros((len(y))))
# preds.index = expression.index
# ncomps=np.arange(1,11)
# sel = model_selection.KFold(n_splits=cv, shuffle=True)    
# sel_inner = model_selection.KFold(n_splits=cv_inner, shuffle=True)

# for tr_ind, te_ind in sel.split(expression):
#     mse=[]
#     expression_cur = expression.iloc[tr_ind,:]
#     for tr_i, te_i in sel_inner.split(expression_cur):
#         mse.append([])
#         tr_cols = expression.index[tr_ind][tr_i]
#         tr_set = pandas.DataFrame(expression.loc[tr_cols,:], copy=True)
#         tr_y = pandas.DataFrame(report['gradient1']).loc[tr_cols,:]
#         te_cols = expression.index[tr_ind][te_i]
#         te_set = pandas.DataFrame(expression.loc[te_cols,:], copy=True)
#         te_y = pandas.DataFrame(report['gradient1']).loc[te_cols,:]
#         for nc in ncomps:
#             clf = PLSRegression(n_components=nc)
#             pred_y = clf.fit(tr_set,tr_y).predict(te_set).flatten()
#             mse[-1].append(mean_squared_error(te_y,pred_y))
#     mse = np.sum(mse,axis=0)
#     nc = ncomps[np.argmin(mse)]
#     best_comps.append(nc)
#     #refit full model
#     clf = PLSRegression(n_components=nc)
#     tr_cols = expression.index[tr_ind]
#     tr_set = pandas.DataFrame(expression.loc[tr_cols,:], copy=True)
#     tr_y = pandas.DataFrame(report['gradient1']).iloc[tr_ind,:]
#     te_cols = expression.index[te_ind]
#     te_set = pandas.DataFrame(expression.loc[te_cols,:], copy=True)
#     te_y = pandas.DataFrame(report['gradient1']).iloc[te_ind,:]
#     mod = clf.fit(tr_set,tr_y)
#     preds.loc[te_cols,:] = mod.predict(te_set)
#     r.append(stats.pearsonr(mod.predict(te_set).flatten(), te_y)[0])


In [ ]:
# cores = multiprocessing.cpu_count()
# pool = multiprocessing.Pool(processes=cores)

# times = 10
# tasks = [[x] for x in range(1,times+1)]
# print(tasks)
# ans = pool.starmap(Component_eval_10cv,tasks)
# np.save(f'{data_dir}/Component_eval_10cv_{times}times', ans) 


# cores = multiprocessing.cpu_count()
# pool = multiprocessing.Pool(processes=cores)

# times = 20
# tasks = [[x] for x in range(11,times+1)]
# print(tasks)
# ans = pool.starmap(Component_eval_10cv,tasks)
# np.save(f'{data_dir}/Component_eval_10cv_{times}times', ans) 
# cores = multiprocessing.cpu_count()
# pool = multiprocessing.Pool(processes=cores)

# times = 30
# tasks = [[x] for x in range(21,times+1)]
# print(tasks)
# ans = pool.starmap(Component_eval_10cv,tasks)
# np.save(f'{data_dir}/Component_eval_10cv_{times}times', ans) 
# cores = multiprocessing.cpu_count()
# pool = multiprocessing.Pool(processes=cores)

# times = 40
# tasks = [[x] for x in range(31,times+1)]
# print(tasks)
# ans = pool.starmap(Component_eval_10cv,tasks)
# np.save(f'{data_dir}/Component_eval_10cv_{times}times', ans) 
# cores = multiprocessing.cpu_count()
# pool = multiprocessing.Pool(processes=cores)

# times = 50
# tasks = [[x] for x in range(41,times+1)]
# print(tasks)
# ans = pool.starmap(Component_eval_10cv,tasks)
# np.save(f'{data_dir}/Component_eval_10cv_{times}times', ans) 
# cores = multiprocessing.cpu_count()
# pool = multiprocessing.Pool(processes=cores)

# times = 60
# tasks = [[x] for x in range(51,times+1)]
# print(tasks)
# ans = pool.starmap(Component_eval_10cv,tasks)
# np.save(f'{data_dir}/Component_eval_10cv_{times}times', ans) 
# cores = multiprocessing.cpu_count()
# pool = multiprocessing.Pool(processes=cores)

# times = 70
# tasks = [[x] for x in range(61,times+1)]
# print(tasks)
# ans = pool.starmap(Component_eval_10cv,tasks)
# np.save(f'{data_dir}/Component_eval_10cv_{times}times', ans) 
# cores = multiprocessing.cpu_count()
# pool = multiprocessing.Pool(processes=cores)

# times = 80
# tasks = [[x] for x in range(71,times+1)]
# print(tasks)
# ans = pool.starmap(Component_eval_10cv,tasks)
# np.save(f'{data_dir}/Component_eval_10cv_{times}times', ans) 
# cores = multiprocessing.cpu_count()
# pool = multiprocessing.Pool(processes=cores)

# times = 90
# tasks = [[x] for x in range(81,times+1)]
# print(tasks)
# ans = pool.starmap(Component_eval_10cv,tasks)
# np.save(f'{data_dir}/Component_eval_10cv_{times}times', ans) 
# cores = multiprocessing.cpu_count()
# pool = multiprocessing.Pool(processes=cores)

# times = 100
# tasks = [[x] for x in range(91,times+1)]
# print(tasks)
# ans = pool.starmap(Component_eval_10cv,tasks)
# np.save(f'{data_dir}/Component_eval_10cv_{times}times', ans) 

In [ ]:
xpsheets = sorted(glob(f'{data_dir}/Component_eval_10cv_*times.npy'))
df = pd.DataFrame()
for sheet in xpsheets:
    ans = np.load(sheet, allow_pickle=True)
    for x in range(len(ans)):
        print(x)
        preds_cur = ans[x][2]
        df_cur = pd.DataFrame(columns = ["Component_9fold_bymse", "Correlation"])
        df_cur.Component_9fold_bymse = ans[x][0]
        df_cur.Correlation = np.array(ans[x][1])
        df = pd.concat([df, df_cur], axis = 0)

In [ ]:
# cv_strategy=100
# r_nc.index=range(1,cv_strategy+1)
# r_nc_toplot = pd.concat([r_nc.T.iloc[:,x] for x in range(len(r_nc))], axis=0)
r_nc_plot = pd.DataFrame({'cn':df.Component_9fold_bymse, '100*10cv_score':abs(df.Correlation.values)})

In [ ]:
for cn in r_nc_plot.cn.unique():
    print(cn)
    print(np.mean(r_nc_plot.loc[r_nc_plot.cn == cn,'100*10cv_score']))
    print(np.median(r_nc_plot.loc[r_nc_plot.cn == cn,'100*10cv_score']))

In [ ]:
RdBu_200 = cm.get_cmap('RdBu', 200)(np.arange(200))
cm.get_cmap('RdBu', 200)

In [ ]:
fig = plt.figure(figsize=(12,10))
# # 默认主轴图axes是subplot(111)
# my_pal = {cn: "#0E437C" if cn == 6 else "#2E5996" for cn in r_nc_plot.cn.unique()}

my_pal = {cn: RdBu_200[30] if cn == 6 else RdBu_200[185] for cn in r_nc_plot.cn.unique()}
# my_pal = {cn: jet_9[7] if cn == 6 else jet_9[1] for cn in r_nc_plot.cn.unique()}
sns.boxplot(x="cn", y='100*10cv_score', data=r_nc_plot, showfliers=False, palette=my_pal, width = 0.8, showmeans=True, 
            boxprops = { 'edgecolor':'grey'},
            meanprops={"marker":"o", "markersize":"10", "markerfacecolor":"white",  "linewidth":1, "markeredgecolor":"black"},
            medianprops={"color":"grey", "linewidth":2},
            whiskerprops={"color":"grey", "linewidth":2},
            capprops={"color":"grey", "linewidth":2}) # palette="hls"
sns.swarmplot(x="cn", y='100*10cv_score', data=r_nc_plot, color ='gray',size = 4, alpha=0.66)
sns.despine()
plt.ylim(0, 0.8)
plt.yticks([0,0.2,0.4,0.6,0.8], fontsize=20)
plt.xticks(fontsize=20)
plt.xlabel('Component number', fontsize=25)
plt.ylabel("Correlation", fontsize=25)

fig_name = 'Cere_02_PLSR_CV_component_cere2etra_100x10cv_Kris.png'
plt.savefig(os.path.join(fig_dir, fig_name), dpi=500, bbox_inches='tight')
plt.show()


## 3.2 Prediction

In [ ]:
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler


def PLSR_pipeline(in_mtx, y, 
                    clf = linear_model.LassoCV(random_state = 123), 
                    cv_strategy = None, cv = 10, test_gene_num = [100], illustrative = False,
                   sanity_check_style = 'separate', cv_labels = [], reverse_axes = True):
    
    final_outputs = {}
    
    if type(in_mtx) == pandas.core.frame.DataFrame:
        in_mtx = in_mtx.values
    
    
    
    print('performing model cross-validation')
    print('using basic %s-fold cross-validation'%cv)
    #scores = model_selection.cross_val_score(clf, in_mtx, y=y, cv=cv)
    predicted = model_selection.cross_val_predict(clf, in_mtx, y=y, cv=cv, n_jobs=-1)
    observed = y
    try:
        score = abs(stats.pearsonr(predicted, y)[0])
    except:
        score = abs(stats.pearsonr(predicted[:,0], y)[0])
    else:
        score = None


    plt.close()
    plt.scatter(predicted, observed)
    # sns.regplot(predicted, observed, fit_reg=False)
    plt.xlabel('CV model predicted FG along axis')
    plt.ylabel('Actual FG')
    plt.show()
    print('model cv score: r = ',score)
    final_outputs.update({'CV_scores': score})

    
    print('running final model')
    mod = clf.fit(in_mtx, y)
    if not hasattr(mod,'coef_'):
        raise IOError('right now, this pipeline can only accept clf objects with a coef_ attribute')
    final_outputs.update({'final_model': mod})
    scr = mod.score(in_mtx, y) 
    print('final model fit r = ',scr)
    if illustrative:
        plt.close()
        sns.regplot(x=mod.predict(in_mtx), y=y)
        plt.xlabel('Model predicted FG')
        plt.ylabel('Actual FG')
        plt.show()
    final_outputs.update({'betas': mod.coef_})
    
    gene_selections = sanity_check(in_mtx, y, mod.coef_, test_gene_num, 
                                   illustrative, sanity_check_style, reverse_axes)
    final_outputs.update({'gene_selections': gene_selections})
    
    return final_outputs


def sanity_check(in_mtx, y, betas, test_gene_num, illustrative, 
                 sanity_check_style, reverse_axes):
    
    if sanity_check_style == 'separate':
        ascores = []
        pscores = []
    else:
        scores = []
    print('running sanity_check')
    try:
        betas = pandas.Series(betas)
    except:
        betas = pandas.Series(betas[:,0])
    outputs = {}
    for num in test_gene_num:
      
        p_chk = betas.sort_values(ascending=False)[:num].index
        a_chk = betas.sort_values(ascending=False)[-num:].index

        pchk_vals = []
        achk_vals = []
        
        for sample in range(in_mtx.shape[0]):
            to_avg = []
            for gene in p_chk:
                to_avg.append(in_mtx[sample,gene])
            if sanity_check_style == 'model':
                pchk_vals.append(np.mean(np.array(to_avg) * betas.loc[p_chk].values))
            else:
                pchk_vals.append(np.mean(to_avg))
        
        for sample in range(in_mtx.shape[0]):
            to_avg = []
            for gene in a_chk:
                to_avg.append(in_mtx[sample,gene])
            if sanity_check_style == 'separate':
                achk_vals.append(np.mean(to_avg))
            elif sanity_check_style == 'model':
                achk_vals.append(np.mean(np.array(to_avg) * betas.loc[a_chk].values))
            else:
                achk_vals.append(np.mean(to_avg) * -1)

        if sanity_check_style != 'separate':
            chk_vals = np.array(pchk_vals) + np.array(achk_vals)
            
        if sanity_check_style == 'separate':
            pr,pp = stats.pearsonr(pchk_vals, y)
            if illustrative:
                plt.close()
                sns.regplot(x=np.array(pchk_vals), y=y, fit_reg=None)
                if reverse_axes:
                    plt.xlabel('expression of anterior direction genes')
                    plt.ylabel('Actual FG')
                    plt.show()
                    print('anterior %s genes vs. y:  r = %s, p = %s \n\n'%(num, pr, pp))
                else:
                    plt.xlabel('expression of posterior direction genes')
                    plt.ylabel('Actual FG')
                    plt.show()
                    print('posterior %s genes vs. y:  r = %s, p = %s \n\n'%(num, pr, pp))

            ar,ap = stats.pearsonr(achk_vals, y)
            if illustrative:
                plt.close()
                sns.regplot(x=np.array(achk_vals), y=y, fit_reg=None)
                if reverse_axes:
                    plt.xlabel('expression of posterior direction genes')
                    plt.ylabel('Actual FG')
                    plt.show()
                    print('posterior %s genes vs. y:  r = %s, p = %s \n\n'%(num, ar, ap))
                else:
                    plt.xlabel('expression of anterior direction genes')
                    plt.ylabel('Actual FG')
                    plt.show()
                    print('anterior %s genes vs. y:  r = %s, p = %s \n\n'%(num, ar, ap))
            
        else:
            r,p = stats.pearsonr(chk_vals, y)
            if illustrative:
                plt.close()
                sns.regplot(x=np.array(chk_vals), y=y, fit_reg=None)
                plt.xlabel('expression of A-P axis genes')
                if reverse_axes:
                    plt.ylabel('Actual FG')
                else:
                    plt.ylabel('Actual FG')
                plt.show()
                print('posterior and anterior %s genes vs. y:  r = %s, p = %s \n\n'%(
                                                                                num, r, p))
            
        if sanity_check_style == 'separate':
            if reverse_axes:
                ascores.append(pr)
                pscores.append(ar)
            else:
                ascores.append(ar)
                pscores.append(pr)
        else:
            scores.append(r)
        if reverse_axes:
            outputs.update({'posterior_genes_%s'%num: a_chk}) 
            outputs.update({'anterior_genes_%s'%num: p_chk})
        else:
            outputs.update({'posterior_genes_%s'%num: p_chk}) 
            outputs.update({'anterior_genes_%s'%num: a_chk})
            
    if len(test_gene_num) > 1:
        if sanity_check_style == 'separate':
            jnk = pandas.concat([pandas.concat([pandas.Series(test_gene_num), pandas.Series(test_gene_num)], axis = 0),
                  pandas.concat([pandas.Series(ascores), pandas.Series(pscores)], axis = 0),
                  pandas.concat([pandas.Series(['a']*len(test_gene_num)), pandas.Series(['p']*len(test_gene_num))], axis = 0)
                  ],axis = 1)
            jnk.columns = ['num','score','class']
            jnk['num'] = jnk['num'].astype('category')
            plt.close()
            # plt.figure(figsize=(7,7))
            sns.catplot(x='num', y='score', data=jnk, hue = 'class',
                        palette={"a": "g", "p": "m"}, aspect=2, size=5)
            plt.xlabel('Number of top genes')
            plt.ylabel('Prediction score')
            plt.show()
        else:
            jnk = pandas.concat([pandas.Series(test_gene_num), 
                                 pandas.Series(scores)
                                ],axis=1)
            jnk.columns = ['num','score']
            jnk['num'] = jnk['num'].astype('category')

            plt.close()
            fig,ax1 = plt.subplots(1, figsize=(8,10))
            sns.catplot(x='num', y='score', data=jnk, ax=ax1)
            ax1.set(xlabel = 'Number of genes', 
                    ylabel ='Explained variance in \nhippocampus a-p gradient')
            plt.show()

    return outputs

In [ ]:
pls_out = PLSR_pipeline(expression,
                        report['gradient1'], 
                        clf = PLSRegression(n_components=6),
                        cv_strategy='score', 
                        illustrative=True,
                        test_gene_num = [1, 2, 5, 10, 20, 50, 100,  # sanity check
                                              200, 500, 1000, 2000, 5000],
                        sanity_check_style = 'model')

In [ ]:
SCORE = abs(stats.pearsonr(pls_out['final_model'].predict(expression).flatten(), report['gradient1'])[0]) # 0.8221736006902771
EVS_cn6 = explained_variance_score(pls_out['final_model'].predict(expression), report['gradient1']) # 0.5206431591363406

## 3.3 Validation

### 3.3.1 101 * 10cv

In [ ]:
#region
# def Eval_10cv_101times(cv_s):
#     clf = PLSRegression(n_components=6)
#     cv=10
#     y = report['gradient1']
#     preds_cur = pandas.DataFrame(np.zeros((len(y),1)))
#     preds_cur.index = expression.index
#     r_cur = pandas.DataFrame(np.zeros((1,cv)))
#     sel = model_selection.KFold(n_splits=cv, shuffle=True, random_state=(123 * (cv_s+1)))
#     fold=0
#     for tr_ind, te_ind in sel.split(expression):
#         tr_cols = expression.index[tr_ind]
#         tr_set = pandas.DataFrame(expression.loc[tr_cols,:], copy=True)
#         tr_y = pandas.DataFrame(report['gradient1']).iloc[tr_ind,:]
#         te_cols = expression.index[te_ind]
#         te_set = pandas.DataFrame(expression.loc[te_cols,:], copy=True)
#         te_y = pandas.DataFrame(report['gradient1']).iloc[te_ind,:]
#         mod = clf.fit(tr_set,tr_y)
#         preds_cur.loc[te_cols, i] = mod.predict(te_set).flatten()
#         r_cur.loc[:,fold] =stats.pearsonr(te_y.values.flatten(), mod.predict(te_set).flatten())[0]
#         fold += 1
#         print('completed iteration',i+1)
#     return r_cur, preds_cur

# cores = multiprocessing.cpu_count()
# pool = multiprocessing.Pool(processes=cores)

# cv_strategy=101
# tasks = [[x] for x in range(1,cv_strategy)]
# ans = pool.starmap(Eval_10cv_101times,tasks)
# r_101 = pd.concat([pd.DataFrame(ans[x]) for x in range(len(ans))], axis=0)
# r_101.index=range(1,ncs+1)
# r_10.to_csv(os.path.join(data_dir, 'PLS_Component_eval_10cv_test.csv'))

# def Eval_10cv_predicts(cv_times): 
#     y = report['gradient1']
#     preds_10 = pandas.DataFrame(np.zeros((len(y),1)))
#     preds_10.index = expression.index
#     clf = PLSRegression(n_components=6)
#     cv=10
#     sel = model_selection.KFold(n_splits=cv, shuffle=True, random_state=(123 * (cv_times+1)))
#     fold=0
#     for tr_ind, te_ind in sel.split(expression):
#         # print('working on fold',fold)
#         tr_cols = expression.index[tr_ind]
#         tr_set = pandas.DataFrame(expression.loc[tr_cols,:], copy=True)
#         tr_y = pandas.DataFrame(report['gradient1']).iloc[tr_ind,:]
#         te_cols = expression.index[te_ind]
#         te_set = pandas.DataFrame(expression.loc[te_cols,:], copy=True)
#         te_y = pandas.DataFrame(report['gradient1']).iloc[te_ind,:]
#         mod = clf.fit(tr_set,tr_y)
#         preds_10.loc[te_cols, :] = mod.predict(te_set)
#         fold += 1
#     print('completed iteration',i+1)
#     return preds_10

# cores = multiprocessing.cpu_count()
# pool = multiprocessing.Pool(processes=cores)

# cv_strategy=100
# y = report['gradient1']
# preds_100 = pandas.DataFrame(np.zeros((len(y),cv_strategy)))
# preds_100.columns = range(cv_strategy)
# preds_100.index = expression.index

# for cv_s in range(10,110,10):
#     tasks = [[x] for x in range(cv_s-10, cv_s)]
#     print(tasks)
#     ans = pool.starmap(Eval_10cv_predicts,tasks) 
#     preds_100.iloc[:, range(cv_s-10, cv_s)] = pd.concat([pd.DataFrame(ans[x]) for x in range(len(ans))], axis=1)
#endregion

In [ ]:
### 101*10cv
clf = PLSRegression(n_components=6)
cv=10
cv_strategy=101
y = report['gradient1']
score = []
# preds_101 = pandas.DataFrame(np.zeros((len(y),cv_strategy)))
# preds_101.columns = range(cv_strategy)
# preds_101.index = expression.index
# r_101 = pandas.DataFrame(np.zeros((cv,cv_strategy)))
# r_101.columns = range(cv_strategy)

# for i in range(cv_strategy):
#     print(i)
#     sel = model_selection.KFold(n_splits=cv, shuffle=True, random_state=(123 * (i+1)))
#     fold=0
#     for tr_ind, te_ind in sel.split(expression):
#         # print('working on fold',fold)
#         tr_cols = expression.index[tr_ind]
#         tr_set = pandas.DataFrame(expression.loc[tr_cols,:], copy=True)
#         tr_y = pandas.DataFrame(report['gradient1']).iloc[tr_ind,:]
#         te_cols = expression.index[te_ind]
#         te_set = pandas.DataFrame(expression.loc[te_cols,:], copy=True)
#         te_y = pandas.DataFrame(report['gradient1']).iloc[te_ind,:]
#         mod = clf.fit(tr_set,tr_y)
#         preds_101.loc[te_cols, i] = mod.predict(te_set).flatten()
#         r_101.loc[fold,i] = stats.pearsonr(te_y.values.flatten(), mod.predict(te_set).flatten())[0]
#         fold += 1
#     # print('completed iteration',i+1)
# pandas.DataFrame(preds_101).to_csv(os.path.join(data_dir, 'Cere_02_PLS_101*10cv_cere2extra_preds.csv'))
# pandas.DataFrame(r_101).to_csv(os.path.join(data_dir, 'Cere_02_PLS_101*10cv_cere2extra_r_1fold.csv'))

In [ ]:
### Final whole 
plt.close()
fig = plt.figure(figsize=(10,10))
sns.regplot(x=report['gradient1'], y=pls_out['final_model'].predict(expression), ci = 95, 
            scatter_kws={'s': 50, 'linewidths': 1, 'color':Hex_1[1]},
            line_kws={'linewidth':3, 'color':Hex_1[1]}) # 'color':'b'
sns.despine()
plt.xlim(0,1.0)
plt.xticks([0,0.5,1])
plt.xlabel('Actual FG', fontsize = 20)
plt.ylim(-0.3,1.3)
plt.yticks([0,0.5,1])
plt.ylabel('Predicted FG')
r = np.float16(stats.pearsonr(pls_out['final_model'].predict(expression).flatten(),report['gradient1'])[0]) # 0.6682418831405792, 1.751126236923602e-34
plt.text(0.05, 1.0,'r = %s'%(float('%.2g' % r)), fontsize=30)
fig_name = 'Cere_02_PLS_Final_cere2extra.png'
plt.savefig(os.path.join(fig_dir,fig_name), dpi=500, bbox_inches='tight')
plt.show()

In [ ]:
# No large difference between sklearn and matlab default plsr
# Try the plsr model from matlab
from pyls.types.regression import simpls
X = expression.values
Y = report['gradient1'].values.reshape([-1,1])
plsr_true = simpls(X=X, Y=Y, n_components=6, seed=123) # dict_keys(['x_weights', 'x_loadings', 'y_loadings', 'x_scores', 'y_scores', 'x_residuals', 'y_residuals', 'beta', 'pctvar', 'mse', 't2'])
np.sum(plsr_true['pctvar'][1]) # Accumulated explained variance for 6 components X: 0.4162980356606241; Y: 0.6646998385364935
preds = np.dot(np.concatenate((np.ones((317,1)), X), axis=1), plsr_true['beta'])  

plt.close()
fig = plt.figure(figsize=(10,10))
sns.regplot(x=report['gradient1'], y=preds, ci = 95, 
            scatter_kws={'s': 50, 'linewidths': 1, 'color':Hex_1[1]},
            line_kws={'linewidth':3, 'color':Hex_1[1]}) # 'color':'b'
sns.despine()
plt.xlim(0,1.0)
plt.xticks([0,0.5,1])
plt.xlabel('Actual FG', fontsize = 20)
plt.ylim(-0.3,1.3)
plt.yticks([0,0.5,1])
plt.ylabel('Predicted FG', fontsize = 20)
r = np.float16(stats.pearsonr(preds.flatten(),report['gradient1'])[0]) # 0.6682418831405792, 1.751126236923602e-34
# np.float16(stats.pearsonr(preds.flatten(),report['gradient1'])[0])
plt.text(0.05, 1.0, f'r = {r}', fontsize=30)
# fig_name = 'Cere_02_PLS_Final_cere2extra.png'
# plt.savefig(os.path.join(fig_dir,fig_name), dpi=500, bbox_inches='tight')
plt.show()

Brainsmash to test the signifcance of 101*10cv

In [ ]:
# Get the median model
y = report['gradient1']
cv_strategy=101
preds_101 = pandas.read_csv(os.path.join(data_dir,'Cere_02_PLS_101*10cv_cere2extra_preds.csv'), index_col=0)
score_true = []
for i in range(cv_strategy):
    score_true.append(stats.pearsonr(preds_101.iloc[:,i].values, report['gradient1'])[0])
score_df = pandas.DataFrame(score_true)
# model_index = [score_df.sort_values(by=0).index[50], score_df.sort_values(by=0).index[49]]
model_index = score_df.sort_values(by=0).index[50]  # 44

np.float64(stats.pearsonr(preds_101.iloc[:,44], report['gradient1'])[0]), float('%.16g' % np.median(score))

xpsheets_name = f'{data_dir}/Permutation/cere_317samples_genelevel/PLS_Score_BrainSmash_Value_Y_17853_*000_101*10cv_preds_44.csv'
csv_list = sorted(glob(xpsheets_name))
Final_perm_cur = pd.Series()
for f in csv_list:
    # num_id = int(f.split('_')[-1].split('.')[0])
    csv_cur = pandas.read_csv(f, index_col=0)
    value_cur = [stats.pearsonr(report['gradient1'], csv_cur.iloc[:, x])[0] for x in range(len(csv_cur.T))]
    # print(csv_cur)
    Final_perm_cur = pd.concat((Final_perm_cur, pd.Series(value_cur)), axis=0)

In [ ]:
import matplotlib as mpl
RdBu_Hex = [mpl.colors.to_hex(x) for x in np.flipud(cm.get_cmap('RdBu')(np.linspace(0,1,200)))[:,0:3]]
RdBu_Hex = RdBu_Hex[: :-1]   

In [ ]:
### Plot
## r2median
preds_101 = pandas.read_csv(os.path.join(data_dir,'Cere_02_PLS_101*10cv_cere2extra_preds.csv'), index_col=0)
plt.close()
fig = plt.figure(figsize=(12,10))

score = []
for i in range(cv_strategy):
    score.append(stats.pearsonr(preds_101.iloc[:,i].values, report['gradient1'])[0])
jnk = pandas.DataFrame(index = range(cv_strategy), 
                       columns = ['score','iteration'])
jnk.loc[:,'iteration'] = list(range(cv_strategy))
jnk.loc[:,'score'] = score
sns.distplot(score, 15, kde=True, rug=True,
             kde_kws={"color": RdBu_200[185], "lw":3 }, 
             hist_kws={ "color": RdBu_200[185], 'alpha': .7 })
sns.despine()
plt.xlim(0.4, 0.5)
plt.xticks([0.4,0.42,0.44,0.46,0.48,0.50], fontsize=20)
plt.yticks(fontsize=20)
ylim = plt.ylim()
plt.vlines(np.median(score), ylim[0], ylim[1], linestyle='-',
           color=jet_9[0], linewidth=4)
plt.text(np.median(score)-0.03, 33.5,'Median=%s'%(float('%.2g' % np.median(score))), 
         fontsize=30)
plt.ylabel('Density', fontsize=25)
plt.xlabel('R between predicted and true FG1', fontsize=25)
fig_name = 'Cere_02_PLS_101x10cv_cere2extra_r2median.png'
plt.savefig(os.path.join(fig_dir, fig_name),  dpi=500, bbox_inches='tight')
plt.show()

In [ ]:
# Plot the median model
fig = plt.figure(figsize=(12,10))
sns.set_context('paper', font_scale=2.5)
sns.set_style('ticks')
sns.regplot(x=report['gradient1'], y=preds_101.iloc[:,44], fit_reg=True, 
            scatter_kws={'s': 50, 'linewidths': 1, 'color':RdBu_Hex[185]},
            line_kws={'linewidth':4, 'color': RdBu_Hex[195]}) # 'color':'b'
sns.despine()
plt.xlim(0,1.0)
plt.ylim(-0.12,1.1)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('Actual FG', fontsize=25)
plt.ylabel('Predicted FG', fontsize=25)
r = np.float16(stats.pearsonr(preds_101.iloc[:,44], report['gradient1'])[0])
plt.text(0.05, 0.92,'$r=%s, p_{SA}<0.01$'%(float('%.2g' % r)), fontsize=30)
fig_name = 'Cere_02_PLS_101x10cv_cere2extra_model44.png'
plt.savefig(os.path.join(fig_dir, fig_name),  dpi=500, bbox_inches='tight')
plt.show()


In [ ]:
fig = plt.figure(figsize=(12,1.2))
sns.set_context('paper', font_scale=2.5)
sns.set_style('ticks')

a = plt.axes([.73, .14, 0.13, 0.2])
sns.distplot(Final_perm_cur, 15, kde=True, rug=True, #hist=True, kde=False, rug=False,
             kde_kws={"color":RdBu_200[185], "lw":3 }, 
             hist_kws={ "color": RdBu_200[185], 'alpha': .7 })
sns.despine()
ylim = plt.ylim()
plt.vlines(0.4571185433269431, ylim[0], ylim[1], linestyle='-',
           color=jet_9[0], linewidth=3, label='Actural r')
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.ylabel("Frequency", fontsize=12)
plt.text(0.15, 2.2,'Actural r', fontsize=12)

fig_name = 'Cere_02_PLS_101x10cv_cere2extra_model44_permutation.png'
plt.savefig(os.path.join(fig_dir, fig_name),  dpi=500, bbox_inches='tight')
plt.show()

In [ ]:
### Plot
## r2median
preds_101 = pandas.read_csv(os.path.join(data_dir,'Cere_02_PLS_101*10cv_cere2extra_preds.csv'), index_col=0)
plt.close()
fig = plt.figure(figsize=(24,10))

plt.subplot(121)
sns.set_context('poster',font_scale=0.8)
score = []
for i in range(cv_strategy):
    score.append(stats.pearsonr(preds_101.iloc[:,i].values, report['gradient1'])[0])
jnk = pandas.DataFrame(index = range(cv_strategy), 
                       columns = ['score','iteration'])
jnk.loc[:,'iteration'] = list(range(cv_strategy))
jnk.loc[:,'score'] = score
sns.distplot(score, 15, kde=True, rug=True,
             kde_kws={"color": RdBu_200[185], "lw":3 }, 
             hist_kws={ "color": RdBu_200[185], 'alpha': .7 })
sns.despine()
plt.xlim(0.4, 0.5)
plt.xticks([0.4,0.42,0.44,0.46,0.48,0.50], fontsize=20)


plt.yticks(fontsize=20)
ylim = plt.ylim()
plt.vlines(np.median(score), ylim[0], ylim[1], linestyle='-',
           color=RdBu_200[185], linewidth=8)
plt.text(np.median(score)-0.034, 33.5,'Median=%s'%(float('%.2g' % np.median(score))), 
         fontsize=30)
plt.ylabel('Density', fontsize=25)
plt.xlabel('R between predicted and true FG1', fontsize=25)
ax = plt.gca()
ax.spines['bottom'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)
# fig_name = 'Cere_02_PLS_101x10cv_cere2extra_r2median.png'
# plt.savefig(os.path.join(fig_dir, fig_name),  dpi=500, bbox_inches='tight')
# plt.show()

# Plot the median model
# fig = plt.figure(figsize=(10,10))
plt.subplot(122)
sns.set_context('paper', font_scale=0.8)
sns.set_style('ticks')
sns.regplot(x=report['gradient1'], y=preds_101.iloc[:,44], fit_reg=True, 
            scatter_kws={'s': 50, 'linewidths': 1, 'color':RdBu_Hex[185]},
            line_kws={'linewidth':4, 'color': RdBu_Hex[195]}) # 'color':'b'
sns.despine()
plt.xlim(0,1.0,)
plt.ylim(-0.12,1.1)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('Actual FG', fontsize=25)
plt.ylabel('Predicted FG', fontsize=25)
r = np.float16(stats.pearsonr(preds_101.iloc[:,44], report['gradient1'])[0])
plt.text(0.05, 0.92,'$r=%s, p_{SA}<0.01$'%(float('%.2g' % r)), fontsize=30)
ax = plt.gca()
ax.spines['bottom'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)


a = plt.axes([.73, .14, 0.13, 0.2])
sns.distplot(Final_perm_cur, 15, kde=True, rug=True, #hist=True, kde=False, rug=False,
             kde_kws={"color":RdBu_200[185], "lw":3 }, 
             hist_kws={ "color": RdBu_200[185], 'alpha': .7 })
sns.despine()
ylim = plt.ylim()
plt.vlines(0.4571185433269431, ylim[0], ylim[1], linestyle='-',
           color=RdBu_200[185], linewidth=4, label='Actural r')
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.ylabel("Density", fontsize=12)
plt.text(0.16, 2.2,'Actural r', fontsize=12)

fig_name = 'Cere_02_PLS_101x10cv_cere2extra_r2median+model44.png'
plt.savefig(os.path.join(fig_dir, fig_name),  dpi=700, bbox_inches='tight')
plt.show()



### 3.3.2 Subjects

In [ ]:
predicted = pls_out['final_model'].predict(expression)

jnk = pandas.concat([pandas.Series(map(lambda x: x[0], predicted)),
                     pandas.Series(report['gradient1'].values),
                     pandas.Series(report['donor'].values)], 
                     axis=1)
jnk.columns = ['predicted','observed','Donor']

plt.close()
# plt.figure(figsize=(10,10))
g = sns.lmplot(x='observed', y='predicted', hue='Donor', data=jnk, fit_reg=True, height=10, aspect=1.6,
               scatter_kws = {'s': 66,  'edgecolor': 'none'}, # 'alpha': 0.5,
               line_kws ={'linewidth': 3})  # palette=Hex_1
g.fig.set_size_inches(10,10)
leg = g._legend
leg.set_bbox_to_anchor([0.35,0.75])
# ax.legend(title='Donor', bbox_to_anchor=(1.05, 1))
plt.xlim(0,1.0)
plt.xticks([0,0.5,1])
plt.xlabel('Actual FG', fontsize = 20)
plt.ylim(-0.3,1.3)
plt.yticks([0,0.5,1])
plt.ylabel('Predicted FG', fontsize = 20)
# plt.xticks(range(0,0.6))
fig_name = 'Cere_02_PLS_allsub.png'
plt.savefig(os.path.join(fig_dir,fig_name), 
            dpi=500, bbox_inches='tight')
plt.show()

In [ ]:
# initialize model
clf = PLSRegression(n_components=6)

plt.close()
# plt.figure(figsize=(5,30))
fig,axes= plt.subplots(6,1, figsize=(10,60), gridspec_kw={'height_ratios': [1, 1, 1, 1, 1, 1]})
for i, subject in enumerate(report.donor.unique()):
    Tr_samps = report[report['donor']!=subject].index  # ID samples from training sample (not the donor)
    Te_samps = report[report['donor']==subject].index  # ID samples from the test sample (the donor)
    # Set X and y
    X = expression[expression.index.isin(Tr_samps)]
    FG = pandas.DataFrame(report['gradient1'])
    FG.index = report.index
    y = FG[FG.index.isin(Tr_samps)]
    # get size of training and test
    nX = len(Te_samps)
    nT = len(Tr_samps)
    # run model
    mod = clf.fit(X,y)
    # predict axis location for samples from the left-out donor
    pred = mod.predict(expression[expression.index.isin(Te_samps)])
    
    # plot it
    sns.set_context('poster', font_scale=0.8)
    sns.set_style('ticks')
    sns.regplot(x=FG[FG.index.isin(Te_samps)], y=pred, ax=axes[i], fit_reg=True, 
                scatter_kws={'s': 50, 'linewidths': 1, 'color':Hex_1[1]},
                line_kws={'linewidth':4, 'color':Hex_1[2]})
    sns.despine()
    axes[i].set_title(subject, fontdict = {'size':25})
    axes[i].set_xlim(0, np.max(FG[FG.index.isin(Te_samps)]).values+0.05)
    axes[i].set_xticks([0, 0.2, 0.4, 0.6, 0.8])
    axes[i].set_ylim(np.min(pred)-0.05,np.max(pred)+0.05)
    axes[i].set_yticks([0, 0.2, 0.4, 0.6, 0.8])
    axes[i].set_xlabel('observed', fontdict = {'size':20})
    axes[i].set_ylabel('predicted', fontdict = {'size':20})
    r = np.float16(stats.pearsonr(pred.flatten(), FG[FG.index.isin(Te_samps)])[0])
    # p

    axes[i].text(0.025, np.max(pred)-0.1,'n = %s/%s\nr = %s'%(nT,nX,float('%.2g' % r)), fontsize=20)
plt.savefig(os.path.join(fig_dir,'Cere_02_PLS_allsub_seperate.png'), 
            dpi=500, bbox_inches='tight')

plt.show()

# 4.0 Features

## 4.1 Quick look at the features

In [ ]:
print(np.min(pls_out['betas']))
print(np.max(pls_out['betas']))
# -0.0017709850960827796
# 0.0014797712849072356

In [ ]:
sorted(pls_out['betas'])[50]

In [ ]:
plt.close()
fig,axes= plt.subplots(1,3, figsize=(30,10))
sns.set_context('poster',font_scale=1)
sns.despine()

sns.distplot(pls_out['betas'],kde=False, ax=axes[0], 
             hist_kws={ "color": jet_9[0], 'alpha': .7 })
axes[0].set_xlim(-0.0020,-0.001)
axes[0].set_xticks([-0.002,-0.0015,-0.001])
axes[0].set_ylim(0,25)
xval = sorted(pls_out['betas'])[50]
plt.plot([xval, xval], [0, 100], linewidth=2)

sns.distplot(pls_out['betas'],kde=False, ax=axes[1], 
             hist_kws={ "color": jet_9[0], 'alpha': .7 })
axes[1].set_xlim(-0.0018,0.0016)
axes[1].set_xticks([-0.0015,-0.00075,0,0.00075,0.0015])
axes[1].set_xlabel('PLSR beta')
axes[1].set_ylabel('N genes')


sns.distplot(pls_out['betas'],kde=False, ax=axes[2], 
             hist_kws={ "color": jet_9[0], 'alpha': .7 })
axes[2].set_xlim(0.001,0.0016)
axes[2].set_xticks([0.0010,0.0012,0.0014,0.0016])
axes[2].set_ylim(0,25)
xval = sorted(pls_out['betas'])[-50]
plt.plot([xval, xval], [0, 100], linewidth=2)


plt.savefig(os.path.join(fig_dir,'Cere_02_PLS_Beta_Dist.png'),
            dpi=500, bbox_inches='tight')
plt.show()

In [ ]:
plt.close()
fig = plt.figure(figsize=(15,10))
sns.set_context('poster',font_scale=0.8)

# Major
sns.distplot(pls_out['betas'],kde=False,
             hist_kws={ "color": RdBu_200[185], 'alpha': .7 })
sns.despine()
plt.xlim(-0.0018,0.0016)
plt.xticks([-0.0015,-0.00075,0,0.00075,0.0015], fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('PLSR beta', fontsize=25)
plt.ylabel('Number', fontsize=25)

# left
al = plt.axes([.18, .2, 0.15, 0.4])
sns.distplot(pls_out['betas'],kde=False, 
             hist_kws={ "color": RdBu_200[185], 'alpha': .7 })
plt.xlim(-0.0020,-0.001)
plt.xticks([-0.002,-0.0015,-0.001], fontsize=20)
plt.yticks(fontsize=20)
plt.ylim(0,25)
xval = sorted(pls_out['betas'])[50]
plt.plot([xval, xval], [0, 100], linewidth=4, color=RdBu_200[185])

# right
ar = plt.axes([.7, .2, 0.15, 0.4])
sns.distplot(pls_out['betas'],kde=False, 
             hist_kws={ "color": RdBu_200[185], 'alpha': .7 })
plt.xlim(0.0009,0.0016)
plt.xticks([0.0010,0.0013,0.0016], fontsize=20)
plt.yticks(fontsize=20)
plt.ylim(0,25)
xval = sorted(pls_out['betas'])[-50]
plt.plot([xval, xval], [0, 100], linewidth=4, color=RdBu_200[185])


plt.savefig(os.path.join(fig_dir,'Cere_02_PLS_Beta_Dist.png'),
            dpi=700/24*15, bbox_inches='tight')
plt.show()

In [ ]:
expression.columns[pandas.Series(pls_out['betas'][:,0]).sort_values(ascending=False)[:50].index].tolist()

In [ ]:
expression.columns[pandas.Series(pls_out['betas'][:,0]).sort_values(ascending=True)[:50].index].tolist()

In [ ]:
# MAKE TABLE FOR OVERLEAF
df = pandas.DataFrame(pls_out['betas'])
df.index = expression.columns
df.columns = ['beta']
gene_table = df.sort_values(ascending=False, by = 'beta')
gene_table.to_csv(os.path.join(data_dir,'PLS_Betas_cere2extra_15624.csv'))


## 4.2 Test the signifcance of each beta

brainsmash to refit

In [ ]:
sheetlist = f'{data_dir}/Permutation/cere_317samples_genelevel/PLS_Beta_BrainSmash_Value_Y_17853_*000.csv'
xpsheets = sorted(glob(sheetlist))

Beta_perm = pandas.DataFrame()
for sheet in tqdm(xpsheets):
    # fid = int(sheet.split('/')[-1].split('_')[-2])
    Beta_cur= pandas.read_csv(sheet, index_col=0)
    Beta_perm = pandas.concat([Beta_cur, Beta_perm], axis=1)
Beta_perm.index = expression.columns
Beta_perm.columns = range(len(Beta_perm.T))

gene_table['p_perm'] = [abs(Beta_perm.loc[x,:]).ge(abs(gene_table.loc[x,'beta'])).sum()/len(Beta_perm.T) for x in gene_table.index]
# 1/10001 > (0.05/15624), so we just use the value larger than the tru divided by the permutation times

In [ ]:
gene_table['p_perm_fdr0.05'] = gene_table['p_perm']<(0.05/15624)
len(gene_table[gene_table['p_perm_fdr0.05']==True]) # 1024 genes
gene_table['type'] = ''
for x in gene_table.index: 
    if gene_table.loc[x,'beta'] > 0:
        gene_table.loc[x,'type'] = 'pos'
        if (~gene_table.loc[x,'p_perm_fdr0.05']):
            gene_table.loc[x,'type'] = 'nosig'
    if gene_table.loc[x,'beta'] < 0:
        gene_table.loc[x,'type'] = 'neg'
        if (~gene_table.loc[x,'p_perm_fdr0.05']):
            gene_table.loc[x,'type'] = 'nosig'
gene_table['color'] = ''
for x in gene_table.index: 
    if gene_table.loc[x,'beta'] > 0:
        gene_table.loc[x,'color'] = Hex_2[1]
        if (~gene_table.loc[x,'p_perm_fdr0.05']):
            gene_table.loc[x,'color'] = "grey"
    if gene_table.loc[x,'beta'] < 0:
        gene_table.loc[x,'color'] = Hex_1[2]
        if (~gene_table.loc[x,'p_perm_fdr0.05']):
            gene_table.loc[x,'color'] = "grey"
gene_table.to_csv(os.path.join(data_dir,'PLS_Betas_cere2extra_15624_permutation.csv'))

figure

In [ ]:
gene_table=pd.read_csv(f'{data_dir}/PLS_Betas_cere2extra_15624_permutation.csv', index_col=0)
gene_table_sig = gene_table.loc[gene_table.loc[:,'p_perm_fdr0.05'],:] # 0.05 1024

In [ ]:
plt.close()
fig = plt.figure(figsize=(10,10))
sns.set_context('poster',font_scale=0.8)
sns.distplot(gene_table_sig['beta'], 25, hist=True, kde=False,
             hist_kws={ "color": RdBu_200[25], 'alpha': .7, "edgecolor": "white" })
sns.despine()
# plt.xlim(-0.0018,0.015)
plt.xticks([-.002,-.001,0,.001, .002], fontsize=20)
plt.yticks(fontsize=20)
plt.xlabel('PLSR beta', fontsize=25)
plt.ylabel('Number', fontsize=25)
plt.savefig(os.path.join(fig_dir,'Cere_02_PLS_Beta_Dist_sig.png'),
            dpi=700/24*7.5, bbox_inches='tight')
plt.show()

In [ ]:
plt.close()
fig = plt.figure(figsize=(24,10))

plt.subplot(121)
sns.set_context('poster',font_scale=1)

# Major
sns.distplot(pls_out['betas'],kde=False,
             hist_kws={ "color": RdBu_200[185], 'alpha': .7 })
sns.despine()
plt.xlim(-0.0018,0.0016)
plt.xticks([-0.0015,-0.00075,0,0.00075,0.0015], fontsize=20)
plt.xlabel('PLSR beta', fontsize=25)
plt.ylabel('N genes', fontsize=25)

# left
al = plt.axes([.18, .2, 0.15, 0.4])
sns.distplot(pls_out['betas'],kde=False, 
             hist_kws={ "color": RdBu_200[185], 'alpha': .7 })
plt.xlim(-0.0020,-0.001)
plt.xticks([-0.002,-0.0015,-0.001])
plt.ylim(0,25)
xval = sorted(pls_out['betas'])[50]
plt.plot([xval, xval], [0, 100], linewidth=4, color=jet_9[0])

# right
ar = plt.axes([.7, .2, 0.15, 0.4])
sns.distplot(pls_out['betas'],kde=False, 
             hist_kws={ "color": RdBu_200[185], 'alpha': .7 })
plt.xlim(0.0009,0.0016)
plt.xticks([0.0010,0.0013,0.0016])
plt.ylim(0,25)
xval = sorted(pls_out['betas'])[-50]
plt.plot([xval, xval], [0, 100], linewidth=4, color=jet_9[0])


plt.subplot(122)